In [1107]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import DB_USERNAME, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# import warnings
# warnings.filterwarnings("ignore")

In [1108]:
# Create a connection string with the postgreSQL info
database_access = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


In [1109]:
# Establish a connection to the PostgreSQL database 
engine = create_engine(database_access)


In [1110]:
# Write a SQL query to select data from the existing view in PostgreSQL 
query = 'SELECT * FROM "Wines"'


In [1111]:
# Use Pandas to read the data from the SQL query and convert into a DataFrame 
wines_df = pd.read_sql_query(query, engine)


In [1112]:
# Display the Dataframe
wines_df.head()

,id,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality,quality_categorisation
0,4799,White,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,6,1
1,1159,Red,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,5,0
2,315,Red,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,6,1
3,31,Red,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,6,1
4,3010,White,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,5,0


## LEWIS - Machine Learning Code 

In [1113]:
# Clean the dataframe, removing columns that will not be used in the ML model
cleaned_wines_df = wines_df.drop(columns=['id','type', 'quality'])

# Display the clean dataframe
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0


In [1114]:
# Split our preprocessed data into our features and target arrays
X = cleaned_wines_df.drop("quality_categorisation", axis=1)
y = cleaned_wines_df["quality_categorisation"]


In [1115]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Convert labels to float
y_train = y_train.astype(float)
y_test = y_test.astype(float)

# Create a StandardScaler instance and fit it to the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [1116]:
#Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [1117]:
#Train the model
model.fit(X_train, y_train, epochs=100)


Epoch 1/100
143/143 [==============================] - 4s 5ms/step - loss: 0.6856 - accuracy: 0.6347
Epoch 2/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6725 - accuracy: 0.6384
Epoch 3/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6609 - accuracy: 0.6384
Epoch 4/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6562 - accuracy: 0.6384
Epoch 5/100
143/143 [==============================] - 1s 8ms/step - loss: 0.6546 - accuracy: 0.6384
Epoch 6/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6547 - accuracy: 0.6384
Epoch 7/100
143/143 [==============================] - 2s 11ms/step - loss: 0.6542 - accuracy: 0.6384
Epoch 8/100
143/143 [==============================] - 1s 9ms/step - loss: 0.6541 - accuracy: 0.6384
Epoch 9/100
143/143 [==============================] - 1s 8ms/step - loss: 0.6542 - accuracy: 0.6384
Epoch 10/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6541 - accur

In [1118]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 1s 6ms/step - loss: 0.6674 - accuracy: 0.6205
Loss: 0.6674253344535828, Accuracy: 0.620512843132019


In [1119]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [1120]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 3s 5ms/step - loss: 0.6463 - accuracy: 0.6340
Epoch 2/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6287 - accuracy: 0.6362
Epoch 3/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6223 - accuracy: 0.6391
Epoch 4/100
143/143 [==============================] - 1s 10ms/step - loss: 0.6205 - accuracy: 0.6417
Epoch 5/100
143/143 [==============================] - 1s 7ms/step - loss: 0.6162 - accuracy: 0.6483
Epoch 6/100
143/143 [==============================] - 1s 10ms/step - loss: 0.6123 - accuracy: 0.6422
Epoch 7/100
143/143 [==============================] - 1s 6ms/step - loss: 0.6078 - accuracy: 0.6510
Epoch 8/100
143/143 [==============================] - 1s 9ms/step - loss: 0.6024 - accuracy: 0.6571
Epoch 9/100
143/143 [==============================] - 1s 6ms/step - loss: 0.6016 - accuracy: 0.6552
Epoch 10/100
143/143 [==============================] - 1s 8ms/step - loss: 0.5882 - accu

In [1121]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 1s 6ms/step - loss: 0.5293 - accuracy: 0.7303
Loss: 0.5292928218841553, Accuracy: 0.7302564382553101


In [1122]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=200, activation="tanh"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [1123]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 4s 9ms/step - loss: 0.6536 - accuracy: 0.6307
Epoch 2/100
143/143 [==============================] - 2s 11ms/step - loss: 0.6107 - accuracy: 0.6615
Epoch 3/100
143/143 [==============================] - 2s 11ms/step - loss: 0.5930 - accuracy: 0.6686
Epoch 4/100
143/143 [==============================] - 1s 7ms/step - loss: 0.5703 - accuracy: 0.6833
Epoch 5/100
143/143 [==============================] - 1s 9ms/step - loss: 0.5681 - accuracy: 0.6853
Epoch 6/100
143/143 [==============================] - 1s 7ms/step - loss: 0.5563 - accuracy: 0.7005
Epoch 7/100
143/143 [==============================] - 1s 9ms/step - loss: 0.5557 - accuracy: 0.6930
Epoch 8/100
143/143 [==============================] - 1s 9ms/step - loss: 0.5380 - accuracy: 0.7123
Epoch 9/100
143/143 [==============================] - 1s 10ms/step - loss: 0.5466 - accuracy: 0.7040
Epoch 10/100
143/143 [==============================] - 2s 12ms/step - loss: 0.5367 - ac

In [1124]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 1s 4ms/step - loss: 0.5159 - accuracy: 0.7303
Loss: 0.5159063935279846, Accuracy: 0.7302564382553101


In [1125]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer with L2 regularization
model.add(Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh", kernel_regularizer=l2(0.001)))

# Second hidden layer with L2 regularization
model.add(Dense(units=200, activation="tanh", kernel_regularizer=l2(0.001)))

# Output layer
model.add(Dense(units=1, activation="sigmoid", kernel_regularizer=l2(0.001)))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [1126]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 3s 5ms/step - loss: 0.8868 - accuracy: 0.6325
Epoch 2/100
143/143 [==============================] - 1s 7ms/step - loss: 0.7874 - accuracy: 0.6591
Epoch 3/100
143/143 [==============================] - 1s 5ms/step - loss: 0.7411 - accuracy: 0.6673
Epoch 4/100
143/143 [==============================] - 1s 6ms/step - loss: 0.7091 - accuracy: 0.6681
Epoch 5/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6908 - accuracy: 0.6811
Epoch 6/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6507 - accuracy: 0.7035
Epoch 7/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6610 - accuracy: 0.6851
Epoch 8/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6219 - accuracy: 0.7049
Epoch 9/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6113 - accuracy: 0.6991
Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6141 - accura

In [1127]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 1s 3ms/step - loss: 0.6260 - accuracy: 0.6574
Loss: 0.6259665489196777, Accuracy: 0.6574358940124512


In [1128]:
# Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=80, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=40, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [1129]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 3s 3ms/step - loss: 2.0511 - accuracy: 0.5340
Epoch 2/100
143/143 [==============================] - 1s 5ms/step - loss: 0.8869 - accuracy: 0.5879
Epoch 3/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7960 - accuracy: 0.6154
Epoch 4/100
143/143 [==============================] - 1s 5ms/step - loss: 0.7739 - accuracy: 0.6294
Epoch 5/100
143/143 [==============================] - 1s 5ms/step - loss: 0.7583 - accuracy: 0.6380
Epoch 6/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7562 - accuracy: 0.6323
Epoch 7/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7478 - accuracy: 0.6329
Epoch 8/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7392 - accuracy: 0.6354
Epoch 9/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7362 - accuracy: 0.6354
Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7334 - accura

In [1130]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 3ms/step - loss: 0.5731 - accuracy: 0.6713
Loss: 0.5730938911437988, Accuracy: 0.6712820529937744


In [1131]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [1132]:
#Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
455/455 [==============================] - 2s 3ms/step - loss: 0.7494 - accuracy: 0.5802
Epoch 2/100
455/455 [==============================] - 1s 3ms/step - loss: 0.6298 - accuracy: 0.6351
Epoch 3/100
455/455 [==============================] - 1s 3ms/step - loss: 0.6148 - accuracy: 0.6516
Epoch 4/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5998 - accuracy: 0.6708
Epoch 5/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5930 - accuracy: 0.6739
Epoch 6/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5866 - accuracy: 0.6732
Epoch 7/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5832 - accuracy: 0.6734
Epoch 8/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5744 - accuracy: 0.6831
Epoch 9/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5711 - accuracy: 0.6886
Epoch 10/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5663 - accura

In [1133]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 4ms/step - loss: 0.5175 - accuracy: 0.7400
Loss: 0.5175202488899231, Accuracy: 0.7400000095367432


In [1134]:
# Define the model
model = Sequential()

## Input layer
model.add(Dense(7, activation='tanh', input_shape=(X_train_scaled.shape[1],)))

# Hidden layers with dropout and L2 regularization
model.add(Dense(1, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary_crossentropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

In [1135]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping, reduce_lr])

Epoch 1/10
143/143 [==============================] - 4s 7ms/step - loss: 0.7909 - accuracy: 0.6217 - val_loss: 0.7489 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 2/10
143/143 [==============================] - 1s 7ms/step - loss: 0.7360 - accuracy: 0.6325 - val_loss: 0.7205 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 3/10
143/143 [==============================] - 1s 5ms/step - loss: 0.7106 - accuracy: 0.6380 - val_loss: 0.7018 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 4/10
143/143 [==============================] - 1s 5ms/step - loss: 0.6933 - accuracy: 0.6371 - val_loss: 0.6893 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 5/10
143/143 [==============================] - 1s 5ms/step - loss: 0.6785 - accuracy: 0.6389 - val_loss: 0.6812 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 6/10
143/143 [==============================] - 1s 5ms/step - loss: 0.6705 - accuracy: 0.6384 - val_loss: 0.6760 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 7/10
143/143 [==============================] - 1s 5ms/step - lo

In [1136]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 3ms/step - loss: 0.6665 - accuracy: 0.6205
Loss: 0.6664620041847229, Accuracy: 0.620512843132019


## KAI - Correlation Code
##### (please use the [wines_df] DataFrame as it contains the 'quality' column that you will need to calculate the correlations)
